In [ ]:
from datasets import load_dataset
import requests

ds = load_dataset("vdaita/gh-commits-2022-meta")["train"]
ds = ds.select(list(range(1000)))

In [ ]:
def get_patch(row):
    repo_name = row['repos'].split(",")[0].replace("[", "").replace("]", "")
    url = f"https://github.com/{repo_name}/commit/{row['commit']}.patch"
    r = requests.get(url)
    print(url)
    text = r.text
    row["patch_text"] = text
    return row

ds = ds.map(get_patch, num_proc=10)

In [ ]:
ds

In [ ]:
ds[:10]

In [ ]:
ds = ds.filter(lambda row: row["patch_text"].count("diff --git") == 1, num_proc=10) # Isolating commits with only one file change

In [ ]:
ds

In [ ]:
from diff_utils import parse_diff, find_best_match
import tempfile # For some reason this isn't working as I expected
import os
import uuid

def get_fn():
    return str(uuid.uuid4())

def process(row):
    diff_text = row["patch_text"]
    lines = diff_text.split('\n')
    start_extraction = False
    result_lines = []
    repo_name = row['repos'].split(",")[0].replace("[", "").replace("]", "")
    
    minus_filepath = ""
    plus_filepath = ""

    for line in lines:
        if line.startswith('---'):
            start_extraction = True
            minus_filepath = line.replace("---", "").strip()
        if line.startswith('+++'):
            plus_filepath = line.replace("+++", "").strip()
        if start_extraction:
            result_lines.append(line)
    
    relevant_patch = '\n'.join(result_lines)
    parsed_diff = parse_diff(relevant_patch)
    desired_filepath = parsed_diff[0].filepath.strip()[2:]

    patch_filepath = row['commit']+ ".patch"
    patch_file = open(patch_filepath, "w+")
    patch_file.write(relevant_patch)
    patch_file.close()

    file_url = f"https://raw.githubusercontent.com/{repo_name}/{row['commit']}/{desired_filepath}"
    
    r = requests.get(file_url)
    text = r.text

    try:
        contents_filepath = row['commit'] + ".file.txt"
        contents_file = open(contents_filepath, "w+")
        contents_file.write(text)
        contents_file.close()

        os.system(f"patch -R {contents_filepath} {patch_filepath} > /dev/null")

        contents_file = open(contents_filepath, "r")
        old_contents = contents_file.read()
        contents_file.close()

        os.remove(contents_filepath)
        os.remove(patch_filepath)

        row["old_contents"] = old_contents
        row["new_contents"] = text
        row["old_file"] = minus_filepath
        row["new_file"] = plus_filepath
    except Exception:
        print("Error on commit: ", row['commit'])
        row["old_contents"] = ""
        row["new_contents"] = ""
        row["old_file"] = ""
        row["new_file"] = ""
    
    return row


In [ ]:
ds = ds.map(process, num_proc=5)